## Distance to Annotation

In [1]:
## Parameters specific to where your folders are and your data
parameter_file = '../narresubmission.yaml'
import yaml
import sys
import scipy.stats as stats
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

#data_folder = doc['data_folder']
#tissues = doc['tissues'].split(',')
#sys.path.append(doc['pipeline_path'])
ref_fa = doc['GENOME_FA']
#anno_gff=doc['annotation']


import os
from os.path import join
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
from tss.visualize import plot_tss_results 
from tss.visualize.fig_utils import helper_save
## Plotting attributes
sns.set_style("whitegrid", {'axes.grid' : False})

from tss.data import annotation
from tss.visualize.fig_utils import helper_save

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


params {'PREFIX': 'ncbi_pice_103', 'ROOT_DIR': '/data/isshamie/TSS/', 'DATA_DIR': '/data/isshamie/TSS/NCBI_PICR_103_Processed/', 'DATA_PROCESSED_DIR': '/data/isshamie/TSS/TSS/data/processed/', 'SUPPLEMENTAL_DIR': '/data/isshamie/TSS/Analysis/supplemental/', 'PIPELINE': '/data/isshamie/TSS/tss', 'GENOME_DIR': '/data/isshamie/genome', 'GENOME_FA': '/data/isshamie/genome/picr.fa', 'GENOME_GFF3': '/data/isshamie/genome/updated_final_sort.gff3', 'GENOME_GTF': '/data/isshamie/genome/updated_final_sort.gtf', 'TISSUES': 'BMDM1hKLA,BMDMwt,Brain,FemaleReproductive,Heart,Intestine,', 'META_FILE': '/data/isshamie/TSS/TSS/parameters/raw_metaSamples.tsv'}
fig_utils from mplh
/data2/resources/software/homer/v4.11_10242019/bin//


In [29]:
pwd

'/data/isshamie/TSS/Analysis/TSS_code/notebooks/NARresubmission/Figure3'

In [33]:
ref_fa

'/data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna'

## Input

In [2]:
indir = doc["results"]

In [3]:
pwd

'/data/isshamie/TSS/Analysis/TSS_code/notebooks/NARresubmission/Figure3'

In [4]:
genome = "GCF"
tss_start = join(indir, genome,'genome','start_site_mRNA.tsv')

In [5]:
tss_annotation = pd.read_csv(tss_start,sep='\t',index_col=0)
tss_annotation

,Chr,Start,End,Strand,actual_start,ID,Parent,Dbxref,Name,gbkey,...,model_evidence,product,transcript_id,Note,exception,end_range,partial,start_range,inference,Length
XM_027386529.1,NW_020822366.1,722308,722308,-,722308,rna-XM_027386529.1,gene-Bmp4,"GeneID:100754303,Genbank:XM_027386529.1",XM_027386529.1,mRNA,...,Supporting evidence includes similarity to: 29...,bone morphogenetic protein 4%2C transcript var...,XM_027386529.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027386528.1,NW_020822366.1,724610,724610,-,724610,rna-XM_027386528.1,gene-Bmp4,"GeneID:100754303,Genbank:XM_027386528.1",XM_027386528.1,mRNA,...,Supporting evidence includes similarity to: 29...,bone morphogenetic protein 4%2C transcript var...,XM_027386528.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027386531.1,NW_020822366.1,1085435,1085435,+,1085435,rna-XM_027386531.1,gene-Cdkn3,"GeneID:100752216,Genbank:XM_027386531.1",XM_027386531.1,mRNA,...,Supporting evidence includes similarity to: 5 ...,cyclin dependent kinase inhibitor 3%2C transcr...,XM_027386531.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027386533.1,NW_020822366.1,1085744,1085744,+,1085744,rna-XM_027386533.1,gene-Cdkn3,"GeneID:100752216,Genbank:XM_027386533.1",XM_027386533.1,mRNA,...,Supporting evidence includes similarity to: 1 ...,cyclin dependent kinase inhibitor 3%2C transcr...,XM_027386533.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027386534.1,NW_020822366.1,1114413,1114413,-,1114413,rna-XM_027386534.1,gene-Cnih1,"GeneID:103160710,Genbank:XM_027386534.1",XM_027386534.1,mRNA,...,Supporting evidence includes similarity to: 10...,cornichon family AMPA receptor auxiliary prote...,XM_027386534.1,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XM_027434589.1,NW_020824185.1,3893,3893,+,3893,rna-XM_027434589.1,gene-LOC113839146,"GeneID:113839146,Genbank:XM_027434589.1",XM_027434589.1,mRNA,...,Supporting evidence includes similarity to: 3 ...,mothers against decapentaplegic homolog 1-like,XM_027434589.1,NaN,NaN,NaN,True,".,3893",NaN,1
XM_027434591.1,NW_020824186.1,17914,17914,+,17914,rna-XM_027434591.1,gene-LOC113839147,"GeneID:113839147,Genbank:XM_027434591.1",XM_027434591.1,mRNA,...,Supporting evidence includes similarity to: 6 ...,PRAME family member 12-like%2C transcript vari...,XM_027434591.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027434593.1,NW_020824186.1,29432,29432,+,29432,rna-XM_027434593.1,gene-LOC103158952,"GeneID:103158952,Genbank:XM_027434593.1",XM_027434593.1,mRNA,...,Supporting evidence includes similarity to: 2 ...,PRAME family member 12-like%2C transcript vari...,XM_027434593.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027434595.1,NW_020824188.1,2147,2147,+,2147,rna-XM_027434595.1,gene-LOC113839149,"GeneID:113839149,Genbank:XM_027434595.1",XM_027434595.1,mRNA,...,Supporting evidence includes similarity to: 25...,zinc finger protein 431-like%2C transcript var...,XM_027434595.1,NaN,NaN,NaN,NaN,NaN,NaN,1


## Output

In [6]:
outdir = doc['outdir']
if not os.path.exists(outdir):
    os.mkdir(outdir)

# if not os.path.exists("distance_to_anno"):
#     os.mkdir("distance_to_anno")
dist_to_anno_dir = join(outdir, "distance_to_anno", genome)
if not os.path.exists(dist_to_anno_dir):
    os.makedirs(dist_to_anno_dir)

#dist_to_anno_dir = join(dist_to_anno_dir, "distance")

In [8]:
len(tss_annotation["gene"].unique())

21387

# Using Output TSS1.exp.bed

In [9]:
peaks_file = join(indir, genome, "eTSS/TSS.exp.bed")
meta_file = join(indir, genome, "eTSS/TSS.exp.meta.tsv")
meta_df = pd.read_csv(meta_file,index_col=0,sep='\t')

peaks_with_dist = annotation.wrap_distance_to_landmarks(peaks_file, landmark_file=tss_start,
                          main_landmark = 'transcript_id',landmark_cols=[],is_bed=True,
                          output_f=None,is_parallel=True,num_par=12, use_prior=True, meta_f=meta_file)
peaks_with_dist

Mergeing meta
                                    Chr    Start      End      Stat Strand
ID                                                                        
p1@Bmp4_XM_027386529.1   NW_020822366.1   722284   722284  1.779596      -
p2@Bmp4_XM_027386529.1   NW_020822366.1   721675   721675  0.506505      -
p3@Bmp4_XM_027386529.1   NW_020822366.1   721908   721908  0.580925      -
p1@Bmp4_XM_027386528.1   NW_020822366.1   724609   724609  4.074290      -
p1@Cdkn3_XM_027386531.1  NW_020822366.1  1085507  1085507  1.212188      +
                                    Chr    Start      End      Stat Strand  \
p1@Bmp4_XM_027386529.1   NW_020822366.1   722284   722284  1.779596      -   
p2@Bmp4_XM_027386529.1   NW_020822366.1   721675   721675  0.506505      -   
p3@Bmp4_XM_027386529.1   NW_020822366.1   721908   721908  0.580925      -   
p1@Bmp4_XM_027386528.1   NW_020822366.1   724609   724609  4.074290      -   
p1@Cdkn3_XM_027386531.1  NW_020822366.1  1085507  1085507  1.212188    

,Chr,Start,End,Strand,Stat,Transcript,Gene,Nearest TSS,Distance to TSS,isSameStrand
p1@Bmp4_XM_027386529.1,NW_020822366.1,722284,722284,-,1.779596,XM_027386529.1,Bmp4,XM_027386529.1,24.0,True
p2@Bmp4_XM_027386529.1,NW_020822366.1,721675,721675,-,0.506505,XM_027386529.1,Bmp4,XM_027386529.1,633.0,True
p3@Bmp4_XM_027386529.1,NW_020822366.1,721908,721908,-,0.580925,XM_027386529.1,Bmp4,XM_027386529.1,400.0,True
p1@Bmp4_XM_027386528.1,NW_020822366.1,724609,724609,-,4.074290,XM_027386528.1,Bmp4,XM_027386528.1,1.0,True
p1@Cdkn3_XM_027386531.1,NW_020822366.1,1085507,1085507,+,1.212188,XM_027386531.1,Cdkn3,XM_027386531.1,72.0,True
...,...,...,...,...,...,...,...,...,...,...
p1@LOC113839076_XM_027434536.1,NW_020824066.1,29526,29526,+,0.667453,XM_027434536.1,LOC113839076,XM_027434536.1,-119.0,True
p1@LOC113839076_XM_027434535.1,NW_020824066.1,29955,29955,+,0.229426,XM_027434535.1,LOC113839076,XM_027434535.1,309.0,True
p1@LOC113839111_XM_027434561.1,NW_020824120.1,10478,10478,+,1.600973,XM_027434561.1,LOC113839111,XM_027434561.1,17.0,True
p2@LOC113839111_XM_027434561.1,NW_020824120.1,9954,9954,+,0.574031,XM_027434561.1,LOC113839111,XM_027434561.1,-507.0,True


In [10]:
# peaks_file = join(genome, "eTSS/TSS.exp.bed")
# meta_file = join(genome, "eTSS/TSS.exp.meta.tsv")
# meta_df = pd.read_csv(meta_file,index_col=0,sep='\t')

# peaks_with_dist = annotation.wrap_distance_to_landmarks(peaks_file, landmark_file=tss_start,
#                           main_landmark = 'transcript_id',landmark_cols=[],is_bed=True,
#                           output_f=None,is_parallel=True,num_par=12)

In [11]:
peaks_with_dist

,Chr,Start,End,Strand,Stat,Transcript,Gene,Nearest TSS,Distance to TSS,isSameStrand
p1@Bmp4_XM_027386529.1,NW_020822366.1,722284,722284,-,1.779596,XM_027386529.1,Bmp4,XM_027386529.1,24.0,True
p2@Bmp4_XM_027386529.1,NW_020822366.1,721675,721675,-,0.506505,XM_027386529.1,Bmp4,XM_027386529.1,633.0,True
p3@Bmp4_XM_027386529.1,NW_020822366.1,721908,721908,-,0.580925,XM_027386529.1,Bmp4,XM_027386529.1,400.0,True
p1@Bmp4_XM_027386528.1,NW_020822366.1,724609,724609,-,4.074290,XM_027386528.1,Bmp4,XM_027386528.1,1.0,True
p1@Cdkn3_XM_027386531.1,NW_020822366.1,1085507,1085507,+,1.212188,XM_027386531.1,Cdkn3,XM_027386531.1,72.0,True
...,...,...,...,...,...,...,...,...,...,...
p1@LOC113839076_XM_027434536.1,NW_020824066.1,29526,29526,+,0.667453,XM_027434536.1,LOC113839076,XM_027434536.1,-119.0,True
p1@LOC113839076_XM_027434535.1,NW_020824066.1,29955,29955,+,0.229426,XM_027434535.1,LOC113839076,XM_027434535.1,309.0,True
p1@LOC113839111_XM_027434561.1,NW_020824120.1,10478,10478,+,1.600973,XM_027434561.1,LOC113839111,XM_027434561.1,17.0,True
p2@LOC113839111_XM_027434561.1,NW_020824120.1,9954,9954,+,0.574031,XM_027434561.1,LOC113839111,XM_027434561.1,-507.0,True


In [12]:
#peaks_with_dist = peaks_with_dist.merge(meta_df[["Gene","Transcript"]],left_index=True,right_index=True)
peaks_with_dist["Absolute"] = np.abs(peaks_with_dist["Distance to TSS"])
peaks_with_dist

,Chr,Start,End,Strand,Stat,Transcript,Gene,Nearest TSS,Distance to TSS,isSameStrand,Absolute
p1@Bmp4_XM_027386529.1,NW_020822366.1,722284,722284,-,1.779596,XM_027386529.1,Bmp4,XM_027386529.1,24.0,True,24.0
p2@Bmp4_XM_027386529.1,NW_020822366.1,721675,721675,-,0.506505,XM_027386529.1,Bmp4,XM_027386529.1,633.0,True,633.0
p3@Bmp4_XM_027386529.1,NW_020822366.1,721908,721908,-,0.580925,XM_027386529.1,Bmp4,XM_027386529.1,400.0,True,400.0
p1@Bmp4_XM_027386528.1,NW_020822366.1,724609,724609,-,4.074290,XM_027386528.1,Bmp4,XM_027386528.1,1.0,True,1.0
p1@Cdkn3_XM_027386531.1,NW_020822366.1,1085507,1085507,+,1.212188,XM_027386531.1,Cdkn3,XM_027386531.1,72.0,True,72.0
...,...,...,...,...,...,...,...,...,...,...,...
p1@LOC113839076_XM_027434536.1,NW_020824066.1,29526,29526,+,0.667453,XM_027434536.1,LOC113839076,XM_027434536.1,-119.0,True,119.0
p1@LOC113839076_XM_027434535.1,NW_020824066.1,29955,29955,+,0.229426,XM_027434535.1,LOC113839076,XM_027434535.1,309.0,True,309.0
p1@LOC113839111_XM_027434561.1,NW_020824120.1,10478,10478,+,1.600973,XM_027434561.1,LOC113839111,XM_027434561.1,17.0,True,17.0
p2@LOC113839111_XM_027434561.1,NW_020824120.1,9954,9954,+,0.574031,XM_027434561.1,LOC113839111,XM_027434561.1,-507.0,True,507.0


In [13]:
peaks_with_dist[~(peaks_with_dist["Nearest TSS"].isin(meta_df["Transcript"]))]

,Chr,Start,End,Strand,Stat,Transcript,Gene,Nearest TSS,Distance to TSS,isSameStrand,Absolute


## Txn

In [14]:
dist_to_txn = peaks_with_dist[peaks_with_dist['Absolute'] == peaks_with_dist.groupby('Nearest TSS')['Absolute'].transform('min')]
print(dist_to_txn.shape)
dist_to_txn = dist_to_txn[~(dist_to_txn.duplicated(subset="Nearest TSS",keep='first'))]
print(dist_to_txn.shape)
sns.distplot(dist_to_txn['Distance to TSS'],norm_hist=False, kde=False)
plt.ylabel('Number of pc eTSSs')
plt.xlabel('TSS Position relative to previous annotation')

plt.title(f'minimum distance of an eTSS to each NCBI promoter start site\nn={dist_to_txn.shape[0]}')
helper_save(join(dist_to_anno_dir, "minimum_for_each_ncbi_transcript"))
plt.close()

# print("Mean: %s" % (np.mean(dist_to_txn["Distance to TSS"])))
# print("Std: %s" % (np.std(dist_to_txn["Distance to TSS"])))
# print("Mode: %s" % (stats.mode(dist_to_txn["Distance to TSS"])[0][0]))
# print("Median: %s" % (np.median(dist_to_txn["Distance to TSS"])))

# print("Number of TSSs in concordance with Reference", (dist_to_txn["Distance to TSS"]==0).sum())
# print("Number of TSSs less than 10nts from Reference", np.abs((dist_to_txn["Distance to TSS"])<10).sum())
# print("Number of TSSs less than 150nts from Reference", np.abs((dist_to_txn["Distance to TSS"])<150).sum())

# print("Total number of TSSs", len(dist_to_txn))
# print("Absolute Mean: %s" % (np.mean(np.abs(dist_to_txn["Distance to TSS"]))))
# print("Absolute Std: %s" % (np.std(np.abs(dist_to_txn["Distance to TSS"]))))



(20630, 11)
(20627, 11)


In [15]:
sns.distplot(np.abs(dist_to_txn['Distance to TSS']),bins=np.arange(0,1200,25),norm_hist=False, kde=False)
plt.ylabel('Number of pc eTSSs')
plt.xlabel('TSS Position relative to previous annotation')
plt.title(f'Distance of each NCBI promoter start site to the closest eTSS\nN promoters={dist_to_txn.shape[0]}')
helper_save(join(dist_to_anno_dir, "absolute_minimum_for_each_ncbi_transcript"))
plt.close()

In [16]:
dist_to_anno_dir

'output/distance_to_anno/GCF'

## Gene

In [17]:
pwd

'/data/isshamie/TSS/Analysis/TSS_code/notebooks/NARresubmission/Figure3'

In [18]:
dist_to_gene = peaks_with_dist[peaks_with_dist['Absolute'] == peaks_with_dist.groupby('Gene')['Absolute'].transform('min')]
dist_to_gene = dist_to_gene[~(dist_to_gene.duplicated(subset="Gene",keep='first'))]
print(dist_to_gene.shape)

sns.distplot(dist_to_gene['Distance to TSS'],norm_hist=False, kde=False)
plt.title(f"Minimum distance of each NCBI gene's promoter start sites-eTSS pair\nN genes={dist_to_gene.shape[0]}")
plt.ylabel('Number of pc eTSSs')
plt.xlabel('TSS Position relative to previous annotation')

helper_save(join(dist_to_anno_dir, "minimum_for_each_ncbi_gene"))
plt.close()
# print("Mean: %s" % (np.mean(dist_to_gene["Distance to TSS"])))
# print("Std: %s" % (np.std(dist_to_gene["Distance to TSS"])))
# print("Mode: %s" % (stats.mode(dist_to_gene["Distance to TSS"])[0][0]))
# print("Median: %s" % (np.median(dist_to_gene["Distance to TSS"])))

# print("Number of TSSs in concordance with Reference", (dist_to_gene["Distance to TSS"]==0).sum())
# print("Number of TSSs less than 10nts from Reference", np.abs((dist_to_gene["Distance to TSS"])<10).sum())
# print("Number of TSSs less than 150nts from Reference", np.abs((dist_to_gene["Distance to TSS"])<150).sum())

# print("Total number of Genes", len(dist_to_gene))
# print("Absolute Mean: %s" % (np.mean(np.abs(dist_to_gene["Distance to TSS"]))))
# print("Absolute Std: %s" % (np.std(np.abs(dist_to_gene["Distance to TSS"]))))



(15308, 11)


In [19]:
sns.distplot(np.abs(dist_to_gene['Distance to TSS']),bins=np.arange(0,1200,25), norm_hist=False, kde=False)
plt.title(f"Minimum distance of each NCBI gene's promoter start sites-eTSS pair\nN genes={dist_to_gene.shape[0]}")
plt.ylabel('Number of pc eTSSs')
plt.xlabel('TSS Position relative to previous annotation')
helper_save(join(dist_to_anno_dir, "absolute_minimum_for_each_ncbi_gene"))
plt.close()

## mRNA TSS

In [20]:
sns.distplot(peaks_with_dist['Distance to TSS'], norm_hist=False, kde=False)

concordant = peaks_with_dist[peaks_with_dist['Distance to TSS']==0]
plt.ylabel('Number of pc eTSSs')
plt.xlabel('TSS Position relative to previous annotation')
plt.title(f"All protein-coding eTSSs to nearest NCBI promoter start site \nN genes={peaks_with_dist.shape[0]}")
helper_save(join(dist_to_anno_dir, "all_promoter_peaks"))

plt.close()
# print("Mean: %s" % (np.mean(peaks_with_dist["Distance to TSS"])))
# print("Std: %s" % (np.std(peaks_with_dist["Distance to TSS"])))
# print("Mode: %s" % (stats.mode(peaks_with_dist["Distance to TSS"])[0][0]))
# print("Median: %s" % (np.median(peaks_with_dist["Distance to TSS"])))

# print("Number of TSSs in concordance with Reference", (peaks_with_dist["Distance to TSS"]==0).sum())
# print("Number of TSSs in concordance with Reference (off by 1)", (peaks_with_dist["Distance to TSS"]==1).sum())
# print("Number of TSSs in concordance with Reference (off by 2)", (peaks_with_dist["Distance to TSS"]==2).sum())
# print("Number of TSSs in concordance with Reference (off by 1 abs)", (np.abs(peaks_with_dist["Distance to TSS"])==1).sum())
# print("Number of TSSs in concordance with Reference (off by 2 abs)", (np.abs(peaks_with_dist["Distance to TSS"])==1).sum())


# print("Number of TSSs less than 10nts from Reference", np.abs((peaks_with_dist["Distance to TSS"])<10).sum())
# print("Total number of TSSs", len(peaks_with_dist))
# print("Absolute Mean: %s" % (np.mean(np.abs(peaks_with_dist["Distance to TSS"]))))
# print("Absolute Std: %s" % (np.std(np.abs(peaks_with_dist["Distance to TSS"]))))



In [21]:
sns.distplot(np.abs(peaks_with_dist['Distance to TSS']),bins=np.arange(0,1200,25), norm_hist=False, kde=False)
plt.title(f"All protein-coding eTSSs to nearest NCBI promoter start site \nN genes={peaks_with_dist.shape[0]}")
plt.ylabel('Number of pc eTSSs')
plt.xlabel('TSS Position relative to previous annotation')
helper_save(join(dist_to_anno_dir, "absolute_all_promoter_peaks"))
plt.close()

## Make stats page

In [22]:
stats = pd.DataFrame(index=["Total observed mRNA TSS", "Protein coding TSS", "Gene"], 
             columns=["NCBI annotation", "Total", "Mean Distance","Median Distance", "Mode Distance", "Standard Deviation Distance",
                      "Mean absolute Distance","Median absolute Distance", "Mode absolute Distance","Standard Deviation absolute Distance",
                      "Number of TSS in concordance", "Number of TSS <= 10nt", 
                      "Number of TSS <= 150nt", "Number of TSS > 150nt", "Number of TSS > 10nt"])

stats.loc["Gene"] = [np.nan,len(dist_to_gene), 
                     np.mean(dist_to_gene["Distance to TSS"]),
                     np.median(dist_to_gene["Distance to TSS"]),
                     dist_to_gene["Distance to TSS"].mode().values[0],
                     np.std(dist_to_gene["Distance to TSS"]),
                     
                     np.mean(dist_to_gene["Absolute"]),
                     np.median(dist_to_gene["Absolute"]),
                     dist_to_gene["Absolute"].mode().values[0],
                     np.std(dist_to_gene["Absolute"]),
                     
                    len(concordant.groupby("Gene")),
                    np.abs((dist_to_gene["Distance to TSS"])<=10).sum(),
                    np.abs((dist_to_gene["Distance to TSS"])<=150).sum(),
                    np.abs((dist_to_gene["Distance to TSS"])>150).sum(),
                    np.abs((dist_to_gene["Distance to TSS"])>10).sum()
                    ]


stats.loc["Protein coding TSS"] = [np.nan,len(dist_to_txn), 
                     np.mean(dist_to_txn["Distance to TSS"]),
                     np.median(dist_to_txn["Distance to TSS"]),
                     dist_to_txn["Distance to TSS"].mode().values[0],
                     np.std(dist_to_txn["Distance to TSS"]),
                     
                     np.mean(dist_to_txn["Absolute"]),
                     np.median(dist_to_txn["Absolute"]),
                     dist_to_txn["Absolute"].mode().values[0],
                     np.std(dist_to_txn["Absolute"]),
                     
                    len(concordant.groupby("Nearest TSS")),
                    np.abs((dist_to_txn["Distance to TSS"])<=10).sum(),
                    np.abs((dist_to_txn["Distance to TSS"])<=150).sum(),
                    np.abs((dist_to_txn["Distance to TSS"])>150).sum(),
                    np.abs((dist_to_txn["Distance to TSS"])>10).sum()

                    ]


stats.loc["Total observed mRNA TSS"] = [np.nan,len(peaks_with_dist), 
                     np.mean(peaks_with_dist["Distance to TSS"]),
                     np.median(peaks_with_dist["Distance to TSS"]),
                     peaks_with_dist["Distance to TSS"].mode().values[0],
                     np.std(peaks_with_dist["Distance to TSS"]),
                     
                     np.mean(peaks_with_dist["Absolute"]),
                     np.median(peaks_with_dist["Absolute"]),
                     peaks_with_dist["Absolute"].mode().values[0],
                     np.std(peaks_with_dist["Absolute"]),
                     
                    (peaks_with_dist["Distance to TSS"]==0).sum(),
                    np.abs((peaks_with_dist["Distance to TSS"])<=10).sum(),
                    np.abs((peaks_with_dist["Distance to TSS"])<=150).sum(),
                    np.abs((peaks_with_dist["Distance to TSS"])>150).sum(),
                    np.abs((peaks_with_dist["Distance to TSS"])>10).sum()

                    ]
stats

,NCBI annotation,Total,Mean Distance,Median Distance,Mode Distance,Standard Deviation Distance,Mean absolute Distance,Median absolute Distance,Mode absolute Distance,Standard Deviation absolute Distance,Number of TSS in concordance,Number of TSS <= 10nt,Number of TSS <= 150nt,Number of TSS > 150nt,Number of TSS > 10nt
Total observed mRNA TSS,NaN,30760,-89.2967,4,17,396.553,283.875,158,2,290.935,116,16252,25503,5257,14508
Protein coding TSS,NaN,20627,-26.5584,16,17,253.148,146.839,54,2,207.913,116,9453,18655,1972,11174
Gene,NaN,15308,3.35393,19,17,193.33,100.601,40,2,165.128,115,6401,14194,1114,8907


In [23]:
# stats_old_error = pd.DataFrame(index=["Total observed mRNA TSS", "Protein coding TSS", "Gene"], 
#              columns=["NCBI annotation", "Total", "Mean Distance","Median Distance", "Mode Distance", "Standard Deviation Distance",
#                       "Mean absolute Distance","Median absolute Distance", "Mode absolute Distance","Standard Deviation absolute Distance",
#                       "Number of TSS in concordance", "Number of TSS <= 10nt", 
#                       "Number of TSS <= 150nt", "Number of TSS > 150nt", "Number of TSS > 10nt"])

# stats_old_error.loc["Gene"] = [np.nan,len(dist_to_gene), 
#                      np.mean(dist_to_gene["Distance to TSS"]),
#                      np.median(dist_to_gene["Distance to TSS"]),
#                      dist_to_gene["Distance to TSS"].mode().values[0],
#                      np.std(dist_to_gene["Distance to TSS"]),
                     
#                      np.mean(dist_to_gene["Absolute"]),
#                      np.median(dist_to_gene["Absolute"]),
#                      dist_to_gene["Absolute"].mode().values[0],
#                      np.std(dist_to_gene["Absolute"]),
                     
#                     len(concordant.groupby("Gene")),
#                     np.abs(((dist_to_gene["Distance to TSS"])<=10).sum()),
#                     np.abs((dist_to_gene["Distance to TSS"])<=150).sum(),
#                     np.abs((dist_to_gene["Distance to TSS"])>150).sum(),
#                     np.abs((dist_to_gene["Distance to TSS"])>10).sum()
#                     ]


# stats_old_error.loc["Protein coding TSS"] = [np.nan,len(dist_to_txn), 
#                      np.mean(dist_to_txn["Distance to TSS"]),
#                      np.median(dist_to_txn["Distance to TSS"]),
#                      dist_to_txn["Distance to TSS"].mode().values[0],
#                      np.std(dist_to_txn["Distance to TSS"]),
                     
#                      np.mean(dist_to_txn["Absolute"]),
#                      np.median(dist_to_txn["Absolute"]),
#                      dist_to_txn["Absolute"].mode().values[0],
#                      np.std(dist_to_txn["Absolute"]),
                     
#                     len(concordant.groupby("Nearest TSS")),
#                     np.abs(((dist_to_txn["Distance to TSS"])<=10)).sum(),
#                     np.abs((dist_to_txn["Distance to TSS"])<=150).sum(),
#                     np.abs((dist_to_txn["Distance to TSS"])>150).sum(),
#                     np.abs((dist_to_txn["Distance to TSS"])>10).sum()

#                     ]


# stats_old_error.loc["Total observed mRNA TSS"] = [np.nan,len(peaks_with_dist), 
#                      np.mean(peaks_with_dist["Distance to TSS"]),
#                      np.median(peaks_with_dist["Distance to TSS"]),
#                      peaks_with_dist["Distance to TSS"].mode().values[0],
#                      np.std(peaks_with_dist["Distance to TSS"]),
                     
#                      np.mean(peaks_with_dist["Absolute"]),
#                      np.median(peaks_with_dist["Absolute"]),
#                      peaks_with_dist["Absolute"].mode().values[0],
#                      np.std(peaks_with_dist["Absolute"]),
                     
#                     (peaks_with_dist["Distance to TSS"]==0).sum(),
#                     np.abs(((peaks_with_dist["Distance to TSS"])<=10).sum()),
#                     np.abs(((peaks_with_dist["Distance to TSS"])<=150).sum()),
#                     np.abs((peaks_with_dist["Distance to TSS"])>150).sum(),
#                     np.abs((peaks_with_dist["Distance to TSS"])>10).sum()

#                     ]
# stats_old_error

,NCBI annotation,Total,Mean Distance,Median Distance,Mode Distance,Standard Deviation Distance,Mean absolute Distance,Median absolute Distance,Mode absolute Distance,Standard Deviation absolute Distance,Number of TSS in concordance,Number of TSS <= 10nt,Number of TSS <= 150nt,Number of TSS > 150nt,Number of TSS > 10nt
Total observed mRNA TSS,NaN,30760,-89.2967,4,17,396.553,283.875,158,2,290.935,116,16252,25503,5257,14508
Protein coding TSS,NaN,20627,-26.5584,16,17,253.148,146.839,54,2,207.913,116,9453,18655,1972,11174
Gene,NaN,15308,3.35393,19,17,193.33,100.601,40,2,165.128,115,6401,14194,1114,8907


## Add old annotation

In [24]:
tss_df = pd.read_csv(tss_start,sep='\t', index_col=0)
tss_df.shape

(35679, 21)

In [25]:
stats.at["Gene", "NCBI annotation"] = len(tss_df.groupby("Parent"))
stats.at["Protein coding TSS","NCBI annotation"] = tss_df.shape[0]
#stats.at["mRNA","NCBI annotation"] = 46750

stats

,NCBI annotation,Total,Mean Distance,Median Distance,Mode Distance,Standard Deviation Distance,Mean absolute Distance,Median absolute Distance,Mode absolute Distance,Standard Deviation absolute Distance,Number of TSS in concordance,Number of TSS <= 10nt,Number of TSS <= 150nt,Number of TSS > 150nt,Number of TSS > 10nt
Total observed mRNA TSS,NaN,30760,-89.2967,4,17,396.553,283.875,158,2,290.935,116,16252,25503,5257,14508
Protein coding TSS,35679,20627,-26.5584,16,17,253.148,146.839,54,2,207.913,116,9453,18655,1972,11174
Gene,21488,15308,3.35393,19,17,193.33,100.601,40,2,165.128,115,6401,14194,1114,8907


In [26]:
stats.to_csv(join(dist_to_anno_dir, "TSS_distance_stats.csv"))

## Add in ncRNAs